In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm  # Thư viện tuyệt vời cho các backbone
import cv2  # OpenCV để đọc ảnh
from tqdm import tqdm # Thanh tiến trình
import matplotlib.pyplot as plt

# --- LỚP CẤU HÌNH TRUNG TÂM ---
# Quản lý tất cả các siêu tham số (hyperparameters) tại đây
class CFG:
    # --- Đường dẫn (Paths) ---
    # (Hãy điều chỉnh các đường dẫn này cho đúng với môi trường của bạn)
    BASE_PATH = '/kaggle/input/csiro-biomass'
    TRAIN_CSV = os.path.join(BASE_PATH, 'train.csv')
    IMAGE_DIR = os.path.join(BASE_PATH, 'train')
    
    # --- Cài đặt Mô hình ---
    MODEL_NAME = 'convnext_tiny' # Bạn có thể đổi sang 'resnet50'
    PRETRAINED = True
    IMG_SIZE = 768 # Kích thước ảnh đầu vào (từ 1000x1000 nén xuống)
    
    # --- Cài đặt Huấn luyện ---
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    BATCH_SIZE = 2   # Bắt đầu với số nhỏ (vì 2 luồng ảnh 768x768 sẽ tốn VRAM)
    EPOCHS = 30
    LEARNING_RATE = 1e-4      # LR cho Giai đoạn 1 (huấn luyện heads)
    
    # --- Cài đặt Giai đoạn 2 (Fine-tuning) ---
    FREEZE_EPOCHS = 15         # Số epochs chỉ huấn luyện 'heads'
    FINETUNE_LR = 1e-5        # LR thấp hơn cho Giai đoạn 2 (toàn bộ mô hình)
    
    NUM_WORKERS = 2  # Số luồng tải dữ liệu
    
    # --- Mục tiêu & Loss ---
    TARGET_COLS = ['Dry_Total_g', 'GDM_g', 'Dry_Green_g']
    LOSS_WEIGHTS = {
        'total_loss': 0.5,
        'gdm_loss': 0.2,
        'green_loss': 0.1
    }
    # 5 MỤC TIÊU ĐỂ TÍNH ĐIỂM (theo đúng thứ tự của cuộc thi)
    ALL_TARGET_COLS = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
    
    # Trọng số R2 (theo thứ tự của ALL_TARGET_COLS)
    R2_WEIGHTS = [0.1, 0.1, 0.1, 0.2, 0.5]

# --- In ra để xác nhận ---
print(f"Sử dụng thiết bị: {CFG.DEVICE}")
print(f"Backbone mô hình: {CFG.MODEL_NAME}")
print(f"Kích thước ảnh huấn luyện: {CFG.IMG_SIZE}x{CFG.IMG_SIZE}")

In [ ]:
# --- Tải dữ liệu và Pivot ---

print(f"Đang tải {CFG.TRAIN_CSV}...")
try:
    # 1. Đọc file CSV gốc (dạng long)
    df_long = pd.read_csv(CFG.TRAIN_CSV)
    print(f"Bảng 'long' gốc có {len(df_long)} hàng.")
    
    # 2. Thực hiện Pivot
    # Chúng ta dùng 'image_path' làm chỉ mục (index) duy nhất
    # Xoay cột 'target_name' thành các cột mới
    # Lấy giá trị từ cột 'target'
    df_wide = df_long.pivot(
        index='image_path',
        columns='target_name',
        values='target'
    )
    
    # 3. Dọn dẹp DataFrame
    # Sau khi pivot, 'image_path' trở thành index.
    # 'reset_index()' sẽ biến nó trở lại thành một cột bình thường.
    df_wide = df_wide.reset_index()
    df_wide.columns.name = None # Xóa tên 'target_name' khỏi trục cột
    
    print(f"Đã pivot! Bảng 'wide' mới có {len(df_wide)} hàng (mỗi hàng 1 ảnh).")
    
    # 4. Hiển thị 5 hàng đầu tiên của bảng 'wide' mới
    print("\n--- 5 hàng đầu của df_wide ---")
    print(df_wide.head())
    
    # 5. (Tùy chọn) Kiểm tra xem các cột mục tiêu của chúng ta có ở đó không
    print("\n--- Các cột có trong df_wide ---")
    print(df_wide.columns.tolist())
    
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file {CFG.TRAIN_CSV}")
    print("Vui lòng kiểm tra lại CFG.TRAIN_CSV")
    # Giả lập df_wide nếu không tìm thấy file để các bước sau không bị lỗi
    df_wide = pd.DataFrame(columns=['image_path'] + CFG.TARGET_COLS)

In [ ]:
from albumentations import (
    Compose, 
    Resize, 
    Normalize,
    HorizontalFlip, 
    VerticalFlip,
    RandomRotate90,  # Chỉ xoay 90, 180, 270
    ColorJitter
)

# --- Định nghĩa Augmentations ---

# Chúng ta sẽ dùng pipeline NÀY cho CẢ hai ảnh (trái và phải) MỘT CÁCH ĐỘC LẬP
def get_train_transforms():
    """
    Tăng cường dữ liệu cho huấn luyện.
    Sẽ được áp dụng độc lập cho img_left và img_right.
    """
    return Compose([
        # 1. Tăng cường hình học (Geometric)
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        RandomRotate90(p=0.5), # Tự động xoay 90, 180, hoặc 270

        # 2. Tăng cường màu sắc (Photometric)
        ColorJitter(
            brightness=0.2, 
            contrast=0.2, 
            saturation=0.2, 
            hue=0.1, 
            p=0.75
        ),

        # 3. Chuẩn hóa (Normalize)
        # Giá trị mean/std tiêu chuẩn của ImageNet
        Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
        
        # 4. Nén ảnh và Chuyển sang Tensor
        # (Nén sau khi augment để đảm bảo chất lượng)
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        ToTensorV2()
    ])

def get_valid_transforms():
    """
    Chỉ chuẩn hóa và nén ảnh cho tập validation (không augment).
    """
    return Compose([
        Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
        Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
        ToTensorV2()
    ])

# --- In ra để xác nhận ---
print("Đã định nghĩa các hàm Augmentation.")
print(f"Ảnh huấn luyện sẽ được augment và nén xuống: {CFG.IMG_SIZE}x{CFG.IMG_SIZE}")
print(f"Ảnh validation sẽ chỉ được nén xuống: {CFG.IMG_SIZE}x{CFG.IMG_SIZE}")

In [ ]:
# (Đây là phiên bản MỚI của lớp BiomassDataset)

class BiomassDataset(Dataset):
    """
    Dataset tùy chỉnh cho chiến lược "Hai luồng".
    
    Sẽ trả về:
    (img_left, img_right, train_targets (3), all_targets (5))
    """
    def __init__(self, df, transforms_fn, image_dir, train_target_cols, all_target_cols):
        self.df = df
        self.transforms_fn = transforms_fn
        self.image_dir = image_dir
        
        # Lưu trữ sẵn để truy cập nhanh
        self.image_paths = df['image_path'].values
        # 3 mục tiêu cho việc tính loss
        self.train_targets = df[train_target_cols].values
        # 5 mục tiêu cho việc tính điểm R2
        self.all_targets = df[all_target_cols].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 1. Lấy thông tin
        img_path_suffix = self.image_paths[idx]
        train_target_vals = self.train_targets[idx]
        all_target_vals = self.all_targets[idx]
        
        # 2. Đọc ảnh gốc (2000x1000)
        filename = os.path.basename(img_path_suffix)
        full_path = os.path.join(self.image_dir, filename)
        image = cv2.imread(full_path)
        if image is None:
            raise FileNotFoundError(f"Không thể đọc ảnh: {full_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 3. Cắt (Crop) thành 2 ảnh (Trái và Phải)
        height, width, _ = image.shape
        mid_point = width // 2
        img_left = image[:, :mid_point]
        img_right = image[:, mid_point:]
        
        # 4. Áp dụng Augmentations ĐỘC LẬP
        transforms = self.transforms_fn()
        img_left_tensor = transforms(image=img_left)['image']
        
        # Lấy pipeline MỚI cho ảnh phải (để augment độc lập)
        transforms_2 = self.transforms_fn()
        img_right_tensor = transforms_2(image=img_right)['image']
        
        # 5. Lấy mục tiêu (Targets)
        train_target_tensor = torch.tensor(train_target_vals, dtype=torch.float32)
        all_targets_tensor = torch.tensor(all_target_vals, dtype=torch.float32)
        
        # 6. Trả về
        return img_left_tensor, img_right_tensor, train_target_tensor, all_targets_tensor

In [ ]:
# --- Thử nghiệm Dataset và DataLoader ---

print("\nĐang kiểm tra Dataset & DataLoader...")

try:
    # 1. Tạo một dataset (dùng augmentation của tập train)
    # Lấy 10 hàng đầu tiên của df_wide để thử nghiệm
    test_df_subset = df_wide.head(10) 
    
    train_dataset = BiomassDataset(
        df=test_df_subset,
        transforms_fn=get_train_transforms, # Truyền tên HÀM
        image_dir=CFG.IMAGE_DIR,
        target_cols=CFG.TARGET_COLS
    )
    
    # 2. Tạo DataLoader
    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG.BATCH_SIZE,
        shuffle=True,
        num_workers=CFG.NUM_WORKERS
    )
    
    # 3. Lấy một batch dữ liệu
    print(f"Đang tải 1 batch (batch_size={CFG.BATCH_SIZE})...")
    img_left_batch, img_right_batch, targets_batch = next(iter(train_loader))
    
    # 4. In ra kích thước (shape)
    print("\n--- Kích thước (Shape) của Batch đầu ra ---")
    print(f"  Ảnh trái (Left): {img_left_batch.shape}")
    print(f"  Ảnh phải (Right): {img_right_batch.shape}")
    print(f"  Mục tiêu (Targets): {targets_batch.shape}")
    
    print("\n--- Kích thước mong đợi ---")
    print(f"  Ảnh (Kỳ vọng):   [Batch, Channels, Height, Width] -> [{CFG.BATCH_SIZE}, 3, {CFG.IMG_SIZE}, {CFG.IMG_SIZE}]")
    print(f"  Mục tiêu (Kỳ vọng): [Batch, Num_Targets] -> [{CFG.BATCH_SIZE}, {len(CFG.TARGET_COLS)}]")
    
    # (Tùy chọn) Hiển thị một ảnh
    plt.imshow(img_left_batch[0].permute(1, 2, 0) * 0.229 + 0.485) # Đã chuẩn hóa
    plt.title("Một ảnh mẫu từ Batch (Trái)")
    plt.show()

except Exception as e:
    print(f"\nLỖI khi kiểm tra DataLoader: {e}")
    print("Hãy kiểm tra lại đường dẫn trong CFG và cấu trúc file ảnh.")

In [ ]:
class BiomassModel(nn.Module):
    """
    Mô hình Hai luồng, Ba đầu ra chuyên biệt.
    
    1. Một backbone (ví dụ: ConvNeXt) dùng chung.
    2. Backbone được áp dụng cho img_left và img_right.
    3. Hai vector đặc trưng được ghép (concatenate) lại.
    4. Vector hợp nhất được đưa vào 3 "đầu" MLP riêng biệt.
    """
    def __init__(self, model_name, pretrained, n_targets=3):
        super(BiomassModel, self).__init__()
        
        # 1. Tải Backbone (Thân) dùng chung
        # num_classes=0: Loại bỏ lớp phân loại gốc.
        # global_pool='avg': Thêm lớp Global Average Pooling
        #                      để lấy ra một vector đặc trưng 1D.
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=0,
            global_pool='avg'
        )
        
        # 2. Lấy số chiều đặc trưng (feature dimension) từ backbone
        # Ví dụ: convnext_tiny là 768, resnet50 là 2048
        self.n_features = self.backbone.num_features
        
        # 3. Tính số chiều sau khi hợp nhất (fusion)
        # (features_left + features_right)
        self.n_combined_features = self.n_features * 2
        
        # 4. Định nghĩa Ba (3) "Đầu" MLP riêng biệt
        # Mỗi "đầu" là một mạng nơ-ron nhỏ chuyên biệt
        
        # --- Đầu cho Dry_Total_g ---
        self.head_total = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1) # Đầu ra 1 giá trị
        )
        
        # --- Đầu cho GDM_g ---
        self.head_gdm = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1) # Đầu ra 1 giá trị
        )
        
        # --- Đầu cho Dry_Green_g ---
        self.head_green = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(self.n_combined_features // 2, 1) # Đầu ra 1 giá trị
        )

    def forward(self, img_left, img_right):
        # 1. Chạy Luồng 1 (Trái)
        features_left = self.backbone(img_left) # Shape: [batch, n_features]
        
        # 2. Chạy Luồng 2 (Phải)
        features_right = self.backbone(img_right) # Shape: [batch, n_features]
        
        # 3. Hợp nhất (Fusion)
        # Ghép 2 vector đặc trưng lại
        combined = torch.cat([features_left, features_right], dim=1) # Shape: [batch, n_combined_features]
        
        # 4. Cho qua các "Đầu" riêng biệt
        out_total = self.head_total(combined)
        out_gdm = self.head_gdm(combined)
        out_green = self.head_green(combined)
        
        # 5. Trả về 3 giá trị dự đoán
        # Chúng ta trả về 3 tensor riêng biệt để hàm loss dễ xử lý
        return out_total, out_gdm, out_green

In [ ]:
# --- Thử nghiệm Kiến trúc Mô hình ---
print("\nĐang kiểm tra kiến trúc mô hình...")

try:
    # 1. Tạo một mô hình
    model = BiomassModel(
        model_name=CFG.MODEL_NAME,
        pretrained=False # Không cần tải trọng số, chỉ kiểm tra kiến trúc
    ).to(CFG.DEVICE)
    
    # 2. Tạo dữ liệu giả (dummy data)
    # (Batch size, channels, height, width)
    dummy_left = torch.randn(CFG.BATCH_SIZE, 3, CFG.IMG_SIZE, CFG.IMG_SIZE).to(CFG.DEVICE)
    dummy_right = torch.randn(CFG.BATCH_SIZE, 3, CFG.IMG_SIZE, CFG.IMG_SIZE).to(CFG.DEVICE)
    
    print(f"Đang đưa 2 batch {dummy_left.shape} vào mô hình...")
    
    # 3. Chạy forward pass
    out_total, out_gdm, out_green = model(dummy_left, dummy_right)
    
    # 4. In ra kích thước đầu ra
    print("\n--- Kích thước (Shape) của Đầu ra Mô hình ---")
    print(f"  Đầu ra Total: {out_total.shape}")
    print(f"  Đầu ra GDM:   {out_gdm.shape}")
    print(f"  Đầu ra Green: {out_green.shape}")
    
    print("\n--- Kích thước mong đợi (mỗi đầu ra) ---")
    print(f"  Kỳ vọng: [Batch, 1] -> [{CFG.BATCH_SIZE}, 1]")

except Exception as e:
    print(f"\nLỖI khi kiểm tra mô hình: {e}")

In [ ]:
class WeightedBiomassLoss(nn.Module):
    """
    Hàm loss tổng hợp có trọng số.
    
    Tính toán 3 hàm MSELoss riêng biệt và cộng chúng lại
    với các trọng số từ CFG.
    """
    def __init__(self, loss_weights_dict):
        super(WeightedBiomassLoss, self).__init__()
        # Chúng ta có thể dùng một (1) hàm MSELoss
        # và gọi nó 3 lần
        self.criterion = nn.SmoothL1Loss()
        
        # Lưu trữ các trọng số
        self.weights = loss_weights_dict

    def forward(self, predictions, targets):
        """
        predictions: Một tuple (out_total, out_gdm, out_green) từ mô hình
        targets: Một tensor [batch_size, 3] từ dataloader
        """
        
        # 1. Tách các dự đoán
        pred_total, pred_gdm, pred_green = predictions
        
        # 2. Tách các mục tiêu (ground truth)
        # targets shape là [batch, 3]
        # Cột 0: Dry_Total_g
        # Cột 1: GDM_g
        # Cột 2: Dry_Green_g
        true_total = targets[:, 0].unsqueeze(-1) # Shape [batch, 1]
        true_gdm   = targets[:, 1].unsqueeze(-1) # Shape [batch, 1]
        true_green = targets[:, 2].unsqueeze(-1) # Shape [batch, 1]
        
        # 3. Tính 3 hàm loss riêng biệt
        loss_total = self.criterion(pred_total, true_total)
        loss_gdm   = self.criterion(pred_gdm, true_gdm)
        loss_green = self.criterion(pred_green, true_green)
        
        # 4. Áp dụng trọng số và tính tổng
        total_loss = (
            self.weights['total_loss'] * loss_total +
            self.weights['gdm_loss'] * loss_gdm +
            self.weights['green_loss'] * loss_green
        )
        
        return total_loss

In [ ]:
# --- Khởi tạo và Kiểm tra Hàm Loss ---

print("\nĐang khởi tạo hàm loss...")

# 1. Khởi tạo
criterion = WeightedBiomassLoss(loss_weights_dict=CFG.LOSS_WEIGHTS)
criterion.to(CFG.DEVICE)

print(f"Hàm loss đã được tạo với các trọng số: {CFG.LOSS_WEIGHTS}")

# 2. Kiểm tra
print("\nĐang kiểm tra hàm loss...")
try:
    # Tạo dữ liệu giả
    # (Giống như output của mô hình)
    dummy_preds = (
        torch.randn(CFG.BATCH_SIZE, 1).to(CFG.DEVICE),
        torch.randn(CFG.BATCH_SIZE, 1).to(CFG.DEVICE),
        torch.randn(CFG.BATCH_SIZE, 1).to(CFG.DEVICE)
    )
    
    # (Giống như output của dataloader)
    dummy_targets = torch.randn(CFG.BATCH_SIZE, len(CFG.TARGET_COLS)).to(CFG.DEVICE)
    
    # Tính loss
    loss_value = criterion(dummy_preds, dummy_targets)
    
    print(f"  Dữ liệu giả: {CFG.BATCH_SIZE} mẫu")
    print(f"  Giá trị loss tính được: {loss_value.item():.4f}")
    print("✓ Hàm loss hoạt động chính xác.")
    
except Exception as e:
    print(f"\nLỖI khi kiểm tra hàm loss: {e}")

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

# --- Thêm vào lớp CFG ---
CFG.N_FOLDS = 5  # Chúng ta sẽ dùng 5-Fold Cross-Validation
CFG.RANDOM_STATE = 42 # Để đảm bảo kết quả có thể tái lập

print(f"\nĐang chuẩn bị {CFG.N_FOLDS}-Fold Cross-Validation...")

# 1. Tạo một cột 'fold' mới trong df_wide, mặc định là -1
df_wide['fold'] = -1

# 2. Tạo Bins (Nhóm) cho mục tiêu quan trọng nhất (Dry_Total_g)
# 'pd.cut' sẽ chia các giá trị liên tục thành 10 nhóm
# (q=10: quantile-based, đảm bảo mỗi nhóm có số lượng mẫu tương đương)
num_bins = int(np.floor(1 + np.log2(len(df_wide)))) # Công thức Sturges (hoặc dùng 10)
if len(df_wide) > 100: # Dùng 10 bins nếu đủ dữ liệu
    num_bins = 10
    
print(f"Sử dụng {num_bins} bins để phân tầng (stratify) trên 'Dry_Total_g'")

df_wide['total_bin'] = pd.cut(
    df_wide['Dry_Total_g'], 
    bins=num_bins, 
    labels=False # Chỉ cần nhãn số
)

# 3. Khởi tạo StratifiedKFold
# Chúng ta chia dựa trên các 'bins' vừa tạo
skf = StratifiedKFold(
    n_splits=CFG.N_FOLDS, 
    shuffle=True, 
    random_state=CFG.RANDOM_STATE
)

# 4. Gán số 'fold' cho mỗi hàng
# 'skf.split' trả về (train_indices, valid_indices)
# Chúng ta chỉ cần valid_indices để gán số fold
for fold_num, (train_idx, valid_idx) in enumerate(skf.split(df_wide, df_wide['total_bin'])):
    # Gán số fold (0, 1, 2, 3, 4) cho các hàng trong tập validation
    df_wide.loc[valid_idx, 'fold'] = fold_num

# 5. Kiểm tra kết quả
print("\n--- Phân phối số lượng mẫu trong mỗi Fold ---")
print(df_wide['fold'].value_counts().sort_index())

print("\n--- df_wide sau khi thêm cột 'fold' (5 hàng đầu) ---")
print(df_wide.head())

In [ ]:
from sklearn.metrics import r2_score

def calculate_competition_score(all_preds_3, all_targets_5):
    """
    Tính điểm R^2 có trọng số của cuộc thi.
    
    Args:
        all_preds_3 (dict): Dict chứa 3 mảng numpy dự đoán từ model.
        all_targets_5 (np.array): Mảng [N, 5] chứa 5 giá trị ground truth.
    
    Returns:
        float: Điểm R^2 cuối cùng.
    """
    
    # 1. Tái cấu trúc 5 dự đoán từ 3 đầu ra
    pred_total = all_preds_3['total']
    pred_gdm = all_preds_3['gdm']
    pred_green = all_preds_3['green']
    
    # Đảm bảo không có giá trị âm
    pred_clover = np.maximum(0, pred_gdm - pred_green)
    pred_dead = np.maximum(0, pred_total - pred_gdm)
    
    # Ghép 5 dự đoán (phải đúng thứ tự của CFG.ALL_TARGET_COLS)
    # ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
    y_preds = np.stack([
        pred_green,
        pred_dead,
        pred_clover,
        pred_gdm,
        pred_total
    ], axis=1) # Shape: [N, 5]
    
    y_true = all_targets_5 # Shape: [N, 5]

    # 2. Tính R^2 cho từng mục tiêu
    # (r2_score của sklearn có thể xử lý multi-output)
    r2_scores = r2_score(y_true, y_preds, multioutput='raw_values')
    
    # 3. Áp dụng trọng số
    weighted_r2_total = 0.0
    for i, weight in enumerate(CFG.R2_WEIGHTS):
        weighted_r2_total += r2_scores[i] * weight
        
    return weighted_r2_total

In [ ]:
import torch.optim as optim
from tqdm import tqdm
import time
import gc

def train_one_epoch(model, loader, criterion, optimizer, device):
    """
    Chạy 1 epoch huấn luyện.
    (Đã cập nhật để xử lý 4 đầu ra từ Dataset)
    """
    model.train()  # Chuyển mô hình sang chế độ .train()
    epoch_loss = 0.0
    
    pbar = tqdm(loader, desc="Training", leave=False)
    
    # *** THAY ĐỔI CHÍNH Ở ĐÂY ***
    # Dataloader giờ trả về 4 giá trị.
    # Chúng ta dùng dấu gạch dưới (_) để bỏ qua giá trị thứ 4 (all_targets).
    for (img_left, img_right, train_targets, _all_targets_ignored) in pbar:
        
        # 1. Chuyển 3 dữ liệu cần thiết sang device (GPU/CPU)
        img_left = img_left.to(device)
        img_right = img_right.to(device)
        targets = train_targets.to(device) # Chỉ 3 mục tiêu cho loss
        
        # 2. Xóa gradients cũ
        optimizer.zero_grad()
        
        # 3. Forward pass (Lan truyền tiến)
        predictions = model(img_left, img_right)
        
        # 4. Tính toán Loss (với 3 mục tiêu)
        loss = criterion(predictions, targets)
        
        # 5. Backward pass (Lan truyền ngược)
        loss.backward()
        
        # 6. Cập nhật trọng số
        optimizer.step()
        
        epoch_loss += loss.item()
        pbar.set_postfix(loss=f'{loss.item():.4f}') # Hiển thị loss hiện tại
        
    # Trả về loss trung bình của epoch
    return epoch_loss / len(loader)

def validate_one_epoch(model, loader, criterion, device):
    """
    Chạy 1 epoch đánh giá VÀ tính điểm R^2.
    """
    model.eval()
    epoch_loss = 0.0
    
    # List để thu thập tất cả dự đoán và nhãn
    all_preds_3 = {'total': [], 'gdm': [], 'green': []}
    all_targets_list = []

    with torch.no_grad():
        pbar = tqdm(loader, desc="Validating", leave=False)
        # Chú ý: Dataloader giờ trả về 4 giá trị
        for (img_left, img_right, train_targets, all_targets) in pbar:
            
            img_left = img_left.to(device)
            img_right = img_right.to(device)
            train_targets = train_targets.to(device) # 3 mục tiêu cho loss
            # all_targets vẫn ở CPU, chúng ta sẽ xử lý sau
            
            # 1. Forward pass
            pred_total, pred_gdm, pred_green = model(img_left, img_right)
            
            # 2. Tính toán Loss (chỉ dựa trên 3 mục tiêu)
            predictions_tuple = (pred_total, pred_gdm, pred_green)
            loss = criterion(predictions_tuple, train_targets)
            epoch_loss += loss.item()
            
            # 3. Thu thập kết quả để tính R^2
            # Chuyển về CPU và lưu dưới dạng numpy
            all_preds_3['total'].append(pred_total.cpu().numpy())
            all_preds_3['gdm'].append(pred_gdm.cpu().numpy())
            all_preds_3['green'].append(pred_green.cpu().numpy())
            all_targets_list.append(all_targets.cpu().numpy())

    # --- Kết thúc Epoch ---
    
    # 4. Ghép tất cả các batch lại
    # Ghép các list mảng numpy thành một mảng numpy lớn
    preds_dict_np = {
        'total': np.concatenate(all_preds_3['total']).flatten(),
        'gdm':   np.concatenate(all_preds_3['gdm']).flatten(),
        'green': np.concatenate(all_preds_3['green']).flatten()
    }
    targets_np_5 = np.concatenate(all_targets_list) # Shape [N, 5]
    
    # 5. Tính điểm R^2
    competition_score = calculate_competition_score(preds_dict_np, targets_np_5)
    
    avg_epoch_loss = epoch_loss / len(loader)
    
    return avg_epoch_loss, competition_score

In [ ]:
import time
import torch.optim as optim
import gc

def run_training(fold_to_run):
    """
    Hàm chính đã được cập nhật với chiến lược 2 giai đoạn (Freeze/Unfreeze).
    """
    print(f"\n{'='*50}")
    print(f"🚀 BẮT ĐẦU HUẤN LUYỆN FOLD {fold_to_run} (Chiến lược 2 giai đoạn) 🚀")
    print(f"{'='*50}")
    
    start_time = time.time()
    
    # 1. Chia dữ liệu (Giữ nguyên)
    print(f"Đang chia dữ liệu cho Fold {fold_to_run}...")
    train_df = df_wide[df_wide['fold'] != fold_to_run].reset_index(drop=True)
    valid_df = df_wide[df_wide['fold'] == fold_to_run].reset_index(drop=True)
    
    # 2. Tạo Datasets (Giữ nguyên)
    # (Giả sử bạn đang dùng BiomassDataset trả về 4 giá trị)
    train_dataset = BiomassDataset(
        df=train_df, transforms_fn=get_train_transforms, image_dir=CFG.IMAGE_DIR,
        train_target_cols=CFG.TARGET_COLS, all_target_cols=CFG.ALL_TARGET_COLS
    )
    valid_dataset = BiomassDataset(
        df=valid_df, transforms_fn=get_valid_transforms, image_dir=CFG.IMAGE_DIR,
        train_target_cols=CFG.TARGET_COLS, all_target_cols=CFG.ALL_TARGET_COLS
    )
    
    # 3. Tạo DataLoaders (Giữ nguyên)
    train_loader = DataLoader(
        train_dataset, batch_size=CFG.BATCH_SIZE, shuffle=True,
        num_workers=CFG.NUM_WORKERS, pin_memory=True
    )
    valid_loader = DataLoader(
        valid_dataset, batch_size=CFG.BATCH_SIZE * 2, shuffle=False,
        num_workers=CFG.NUM_WORKERS, pin_memory=True
    )
    
    # 4. Khởi tạo Mô hình & Loss (Hỗ trợ Multi-GPU)
    print(f"Đang tải backbone '{CFG.MODEL_NAME}'...")
    model_base = BiomassModel(CFG.MODEL_NAME, CFG.PRETRAINED)
    
    if torch.cuda.device_count() > 1:
        print(f"Sử dụng {torch.cuda.device_count()} GPU với nn.DataParallel.")
        model = nn.DataParallel(model_base)
    else:
        model = model_base
    model.to(CFG.DEVICE)
    
    criterion = WeightedBiomassLoss(CFG.LOSS_WEIGHTS).to(CFG.DEVICE)
    
    # =================================================================
    # ✨ GIAI ĐOẠN 1: HUẤN LUYỆN "HEADS" (ĐÓNG BĂNG BACKBONE)
    # =================================================================
    print(f"\n--- GIAI ĐOẠN 1: Đóng băng Backbone (Training Heads) ---")
    print(f"Epochs: 1 đến {CFG.FREEZE_EPOCHS} | LR: {CFG.LEARNING_RATE}")
    
    # Đóng băng tất cả các tham số của backbone
    # (Dùng .module. để truy cập mô hình gốc bên trong DataParallel)
    for param in model.module.backbone.parameters():
        param.requires_grad = False
        
    # Tạo optimizer CHỈ cho các "heads" (tham số có requires_grad=True)
    optimizer = optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()), 
        lr=CFG.LEARNING_RATE
    )
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.1, patience=2 # Giảm patience
    )
    
    best_score = -float('inf') # Theo dõi điểm R^2 tốt nhất qua CẢ 2 GIAI ĐOẠN

    for epoch in range(1, CFG.FREEZE_EPOCHS + 1):
        print(f"\n--- Epoch {epoch}/{CFG.EPOCHS} (Giai đoạn 1) ---")
        
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, CFG.DEVICE)
        valid_loss, competition_score = validate_one_epoch(model, valid_loader, criterion, CFG.DEVICE)
        
        scheduler.step(valid_loss)
        
        print(f"Epoch {epoch} - Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f} | Score (R^2): {competition_score:.4f}")
        
        if competition_score > best_score:
            best_score = competition_score
            print(f"✨ Score R^2 cải thiện! Đang lưu mô hình 'best_model_fold{fold_to_run}.pth'...")
            torch.save(model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict(), 
                       f'best_model_fold{fold_to_run}.pth')

    # =================================================================
    # ✨ GIAI ĐOẠN 2: HUẤN LUYỆN TOÀN BỘ (RÃ ĐÔNG BACKBONE)
    # =================================================================
    print(f"\n--- GIAI ĐOẠN 2: Rã đông Backbone (Fine-tuning) ---")
    print(f"Epochs: {CFG.FREEZE_EPOCHS + 1} đến {CFG.EPOCHS} | LR: {CFG.FINETUNE_LR}")

    # Rã đông toàn bộ mô hình
    for param in model.module.backbone.parameters():
        param.requires_grad = True
        
    # Tạo một optimizer MỚI cho TOÀN BỘ mô hình với LR thấp
    optimizer = optim.Adam(
        model.parameters(), 
        lr=CFG.FINETUNE_LR # Sử dụng LR mới
    )
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.2, patience=3
    )
    
    # Tiếp tục vòng lặp epoch
    for epoch in range(CFG.FREEZE_EPOCHS + 1, CFG.EPOCHS + 1):
        print(f"\n--- Epoch {epoch}/{CFG.EPOCHS} (Giai đoạn 2) ---")
        
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, CFG.DEVICE)
        valid_loss, competition_score = validate_one_epoch(model, valid_loader, criterion, CFG.DEVICE)
        
        scheduler.step(valid_loss)
        
        print(f"Epoch {epoch} - Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f} | Score (R^2): {competition_score:.4f}")
        
        if competition_score > best_score:
            best_score = competition_score
            print(f"✨ Score R^2 cải thiện! Đang lưu mô hình 'best_model_fold{fold_to_run}.pth'...")
            torch.save(model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict(), 
                       f'best_model_fold{fold_to_run}.pth')
            
    # --- KẾT THÚC HUẤN LUYỆN ---
    end_time = time.time()
    print(f"\n🎉 Hoàn thành Fold {fold_to_run} sau {(end_time - start_time)/60:.2f} phút.")
    print(f"Điểm R^2 tốt nhất: {best_score:.4f}")
    
    # Dọn dẹp
    del model, train_loader, valid_loader, train_dataset, valid_dataset
    gc.collect()
    torch.cuda.empty_cache()

# --- BẮT ĐẦU CHẠY HUẤN LUYỆN ---
# (Giữ nguyên)
try:
    for i in range(CFG.N_FOLDS):
        run_training(fold_to_run=i)
except Exception as e:
    gc.collect()
    torch.cuda.empty_cache()
    raise e